In [1]:
import tensorflow as tf
from tensorflow import keras

2022-04-08 11:04:17.287959: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/benson516/agv_ws/devel/lib:/opt/ros/noetic/lib
2022-04-08 11:04:17.288036: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Managing the GPU RAM

In [2]:
'''
$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_1.py
'''

'\n$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py\n$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_1.py\n'

In [3]:
# # Set memory limit to GPUs
# for gpu in tf.config.experimental.list_physical_devices("GPU"):
#     tf.config.experimental.set_virtual_device_configuration(
#         gpu,
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

In [4]:
# # Runtime grow memory allocation
# for gpu in tf.config.experimental.list_physical_devices("GPU"):
#     tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
# Split into virtual GPUs
physical_gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_virtual_device_configuration(
            physical_gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
             tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

2022-04-08 11:04:18.848395: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-04-08 11:04:21.898797: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-08 11:04:21.898924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (benson516-P1G2): /proc/driver/nvidia/version does not exist


IndexError: list index out of range

## Placing Operations and Variables on Devices

In [6]:
a = tf.Variable(42.0)

2022-04-08 11:04:25.199945: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
a.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [8]:
b = tf.Variable(42)

In [9]:
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [10]:
# Set the debug log to show the device placement
tf.debugging.set_log_device_placement(True)

In [11]:
c = tf.Variable(42.0)

Place on other device

In [12]:
with tf.device("/cpu:0"):
    c = tf.Variable(42.0)

In [13]:
c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

# Training Models Across Multiple Devices
## Training at Scale Using the Distribution Strategies API

Prepare Data

In [15]:
import numpy as np
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

Create the model in the distribution scope

In [18]:
distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    mirrored_model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28, 1]),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    mirrored_model.compile(loss="sparse_categorical_crossentropy",
                           optimizer=keras.optimizers.SGD(lr=1e-2),
                           metrics=["accuracy"])
    
batch_size = 100 # must be divisible by the number of replicas
history = mirrored_model.fit(X_train, y_train, epochs=10, batch_size=batch_size)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2022-04-11 10:07:51.018914: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_35452"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto_shard_policy = Aut

Epoch 1/10
550/550 [==============================] - 3s 3ms/step - loss: 1.1221 - accuracy: 0.7315
Epoch 2/10
550/550 [==============================] - 1s 2ms/step - loss: 0.5140 - accuracy: 0.8720
Epoch 3/10
550/550 [==============================] - 1s 2ms/step - loss: 0.4184 - accuracy: 0.8880
Epoch 4/10
550/550 [==============================] - 1s 2ms/step - loss: 0.3761 - accuracy: 0.8967
Epoch 5/10
550/550 [==============================] - 1s 2ms/step - loss: 0.3504 - accuracy: 0.9026
Epoch 6/10
550/550 [==============================] - 1s 2ms/step - loss: 0.3322 - accuracy: 0.9063
Epoch 7/10
550/550 [==============================] - 1s 2ms/step - loss: 0.3181 - accuracy: 0.9100
Epoch 8/10
550/550 [==============================] - 1s 2ms/step - loss: 0.3063 - accuracy: 0.9136
Epoch 9/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2962 - accuracy: 0.9161
Epoch 10/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2871 - accuracy: 0.9188

In [25]:
# --- New method (can be load by keras)
# keras.models.save_model(mirrored_model, "my_mnist_model.h5")
mirrored_model.save("my_mnist_model.h5")  # The model will be saved as a normal model, no distribution information is kept.

In [26]:
# Load a model as a distributed model
with distribution.scope():
    mirrored_model = keras.models.load_model("my_mnist_model.h5")

In [27]:
# Use a subset of GPUs
distribution = tf.distribute.MirroredStrategy(["/gpu:0", "/gpu:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [29]:
# Use centralized parameter method instead of the mirrored strategy
distribution = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'


In [31]:
# Load a model as a distributed model
with distribution.scope():
    mirrored_model = keras.models.load_model("my_mnist_model.h5")
    
batch_size = 100 # must be divisible by the number of replicas
history = mirrored_model.fit(X_train, y_train, epochs=10, batch_size=batch_size)

Epoch 1/10
  1/550 [..............................] - ETA: 1:13 - loss: 0.3565 - accuracy: 0.8900

2022-04-11 12:02:31.621973: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_48372"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 100
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto_shard_policy = Aut

550/550 [==============================] - 1s 2ms/step - loss: 0.2787 - accuracy: 0.9212
Epoch 2/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2710 - accuracy: 0.9241
Epoch 3/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2639 - accuracy: 0.9261
Epoch 4/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2572 - accuracy: 0.9279
Epoch 5/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2509 - accuracy: 0.9299
Epoch 6/10
550/550 [==============================] - 2s 3ms/step - loss: 0.2447 - accuracy: 0.9318
Epoch 7/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2389 - accuracy: 0.9336
Epoch 8/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2335 - accuracy: 0.9357
Epoch 9/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2281 - accuracy: 0.9368
Epoch 10/10
550/550 [==============================] - 1s 2ms/step - loss: 0.2230 - accuracy: 0.9386


## Training a Model on a TensorFlow Cluster

In [32]:
# Cluster spec.
cluster_spec = {
    "worker": [
        "machine-a.examplecom:2222",  # /job:worker/task:0
        "machine-b.example.com:2222"  # /job:worker/task:1
    ],
    "ps": ["machine-a.example.com:2221"] # /job:ps/task:0
}

The following is an example of starting a task (a process) on one of the machine.

In [38]:
import os 
import json

# Before importing tensorflow, set the "TF_CONFIG" environment variable
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 0}
})

# Then, import tensorflow 
import tensorflow as tf
from tensorflow import keras

On each machine, run the following training code.

In [39]:
distribution = tf.distribute.experimental.MultiWorkerMirroredStrategy()

with distribution.scop():
    mirrored_model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28, 1]),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    mirrored_model.compile(loss="sparse_categorical_crossentropy",
                           optimizer=keras.optimizers.SGD(lr=1e-2),
                           metrics=["accuracy"])
    
batch_size = 100 # must be divisible by the number of replicas
history = mirrored_model.fit(X_train, y_train, epochs=10, batch_size=batch_size)

RuntimeError: Collective ops must be configured at program startup

Train on TPUs

In [40]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialize_tpu_system(resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

ValueError: Please provide a TPU Name to connect to.